# Projet 4 : Anticipez les besoins en consommation électrique de bâtiments

# Import des données

### Chargement des bibliothèques

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
sns.set()

## Collab


In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [5]:
#répertoir avec nos fichiers
images_dir = "/content/drive/My Drive/data/OPC4/"

#Import csv in a DataFrame
data_2015 = pd.read_csv(images_dir +"2015-building-energy-benchmarking.csv")
data_2016 = pd.read_csv(images_dir +"2016-building-energy-benchmarking.csv")

print ('CSV Imported. Shape :')
data_2015.shape 
data_2016.shape
print("Le jeu de données de 2015 contient {} lignes et {} colonnes.".format(data_2015.shape[0],data_2015.shape[1]))
print("Le jeu de données de 2016 contient {} lignes et {} colonnes.".format(data_2016.shape[0],data_2016.shape[1]))

CSV Imported. Shape :
Le jeu de données de 2015 contient 3340 lignes et 47 colonnes.
Le jeu de données de 2016 contient 3376 lignes et 46 colonnes.


# Analyse exploratoire

## **Exploration initiale des données**

In [0]:
def shape(dataframe):
    """Calcuates and displays the shape of the dataframe and the filling rage"""
    
    # get the numbers of rows and columns in the dataframe
    nb_rows, nb_columns = dataframe.shape
    print("\nDataFrame has {} rows and {} columns.".format(nb_rows,nb_columns))

    # get the number of non-Nan data in the dataframe
    nb_data = dataframe.describe(include='all').loc["count"].sum()

    # computing the filling rate by rounding down to 2 decimal places
    import math
    filling_rate = math.floor(nb_data*10000 / (nb_rows * nb_columns))/10000
    print("The global filling rate of the DataFrame is : {:.2%}".format(filling_rate))
    
    # computing the missing values
    nb_missing = int((nb_rows * nb_columns) - nb_data)
    print("There are {} missing values out of {}.".format(nb_missing, int(nb_rows * nb_columns)))

In [0]:
shape(data_2015)
shape(data_2016)


DataFrame has 3340 rows and 47 columns.
The global filling rate of the DataFrame is : 83.11%
There are 26512 missing values out of 156980.

DataFrame has 3376 rows and 46 columns.
The global filling rate of the DataFrame is : 87.15%
There are 19952 missing values out of 155296.


In [7]:
data_2015.head(1)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,1,12.0,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,65.0,78.9,80.3,173.5,175.1,6981428.0,7097539.0,2023032.0,1080307.0,3686160.0,12724.0,1272388.0,0.0,249.43,2.64,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081


In [8]:
data_2016.head(1)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,7,DOWNTOWN,47.6122,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.5,189.0,7226362.5,7456910.0,2003882.0,1156514.25,3946027.0,12764.5293,1276453.0,False,NaN,Compliant,NaN,249.98,2.83


Nous constatons que la plupart des lignes (correspondant à un batîment particulier) se retrouvent à la fois dans les données de 2015 et dans celles de 2016.

L'agrégation des données de 2015 et de 2016 nécessitera donc de gérer les données dupliquées.

In [9]:
print("\nNames of the {} common columns :\n".format(len(set(data_2015)&set(data_2016))), set(data_2015)&set(data_2016))
print("\nNames of the {} columns only in 2015 dataset:\n".format(len(set(data_2015)-set(data_2016))), set(data_2015)-set(data_2016))
print("\nNames of the {} columns only in 2016 dataset:\n".format(len(set(data_2016)-set(data_2015))), set(data_2016)-set(data_2015))


Names of the 37 common columns :
 {'ENERGYSTARScore', 'ListOfAllPropertyUseTypes', 'PropertyGFAParking', 'ComplianceStatus', 'SiteEUI(kBtu/sf)', 'DataYear', 'SecondLargestPropertyUseTypeGFA', 'SourceEUI(kBtu/sf)', 'YearsENERGYSTARCertified', 'LargestPropertyUseType', 'NaturalGas(therms)', 'SteamUse(kBtu)', 'CouncilDistrictCode', 'TaxParcelIdentificationNumber', 'Neighborhood', 'ThirdLargestPropertyUseType', 'YearBuilt', 'ThirdLargestPropertyUseTypeGFA', 'Electricity(kBtu)', 'NumberofFloors', 'SiteEnergyUseWN(kBtu)', 'SiteEnergyUse(kBtu)', 'SecondLargestPropertyUseType', 'NaturalGas(kBtu)', 'PropertyName', 'SiteEUIWN(kBtu/sf)', 'Outlier', 'LargestPropertyUseTypeGFA', 'PropertyGFABuilding(s)', 'PrimaryPropertyType', 'BuildingType', 'SourceEUIWN(kBtu/sf)', 'Electricity(kWh)', 'OSEBuildingID', 'NumberofBuildings', 'PropertyGFATotal', 'DefaultData'}

Names of the 10 columns only in 2015 dataset:
 {'Zip Codes', 'GHGEmissions(MetricTonsCO2e)', 'Location', 'OtherFuelUse(kBtu)', 'City Council 

Nous constatons que certains colonnes spécifiques ont en fait la même sémantique. Par exemple :
* la colonne 'Comment' de 2015 correspond à la colonne 'Comments' de 2016
* la colonne 'Zip Codes' de 2015 correspond à la colonne 'ZipCode' de 2016
* la colonne 'GHGEmissionsIntensity(kgCO2e/ft2)' de 2015 correspond à la colonne 'GHGEmissionsIntensity' de 2016
* la colonne 'GHGEmissions(MetricTonsCO2e)' de 2015 correspond à la colonne 'TotalGHGEmissions' de 2016

In [0]:
# Renaming a column to adopt names of 2016 database
data_2015 = data_2015.rename(columns={
    'GHGEmissions(MetricTonsCO2e)':"TotalGHGEmissions",
    'Comment':'Comments',
    'Zip Codes':'ZipCode',
    'GHGEmissionsIntensity(kgCO2e/ft2)':'GHGEmissionsIntensity',
})

Nous constatons aussi que certains colonnes spécifiques ont été reformatées. Ainsi, la colonne 'Location' de 2015 a été séparée en plusieurs colonnes en 2016 :
* 'State'
* 'City'
* 'Address'
* 'Longitude'
* 'Latitude'

## Concaténation

In [11]:
data = pd.concat([data_2015, data_2016], sort=False)
shape(data)


DataFrame has 6716 rows and 52 columns.
The global filling rate of the DataFrame is : 76.11%
There are 83420 missing values out of 349232.


## Fusion des lignes correspondant à des bâtiments dupliqués
Nous identifions les bâtiments dupliqués sur la base de l'identifiant 'OSEBuildingID'.

Nous fusionnons ensuite les lignes et supprimons les doublons.

In [0]:
def duplicates_merging(dataframe, keys):
    """This function handles duplicates rows in the dataframe, based on a subset of columns (features)."""

    # Checking if there are some duplicated rows
    duplicates_mask = dataframe.duplicated(subset=keys, keep=False)
    if True not in list(duplicates_mask.unique()):
        print("No duplicates.")
        return dataframe
    
    print("Number of duplicates rows :", len(dataframe[duplicates_mask]))
    
    # Filtering the dataframe to keep only duplicated rows
    duplicates_mask = dataframe.duplicated(subset=keys, keep=False)
    duplicates_df = dataframe[duplicates_mask]

    # Group-by subset of columns used for key, sort=False to speed-up
    gb = duplicates_df.groupby(keys, sort=False)

    # Initializing aggregated dataframe
    agg_df = pd.DataFrame()

    # Identification of numerical and non-numerical columns
    numeric_columns = list(dataframe.select_dtypes(include=[np.number]).columns.values)

    # defining aggregation function for non-numerical columns
    def agg_mode(x): m = pd.Series.mode(x); return m.values[0] if not m.empty else np.nan

    # Iterating upon columns
    for column in dataframe.columns:

        # Calculate the mean of each group for numeric columns
        if column in numeric_columns:
            agg_col = gb[column].agg('mean')

        # Calculate the mode of each group for numeric columns
        else:
            agg_col = gb[column].agg(agg_mode)
            
        # adding the aggregated column to aggregated dataframe
        agg_df = pd.concat([agg_df, agg_col], axis=1, sort=True)

    # Dropping all duplicates
    dataframe_cleaned = dataframe.drop_duplicates(subset=keys, keep=False)

    # Concatenating the dataframe without duplicates and the aggregated rows for duplicates
    result = dataframe_cleaned.append(agg_df, ignore_index=True, sort=False)

    print("Shape after handling duplicates :", result.shape)
    return result

In [13]:
%%capture --no-stdout --no-display

dataframe = data
keys = ['OSEBuildingID']

data = duplicates_merging(dataframe, keys)
shape(data)

Number of duplicates rows : 6568
Shape after handling duplicates : (3432, 52)

DataFrame has 3432 rows and 52 columns.
The global filling rate of the DataFrame is : 84.76%
There are 27184 missing values out of 178464.


## Retypage des variables catégorielles

In [0]:
# Select columns of object dtypes
obj_columns = list(data.select_dtypes(include='object').columns)

# Convert object dtypes features as categorical data ('category' dtypes)
data[obj_columns] = data[obj_columns].astype('category')

## Suppression des données où la cible est manquante

Certaines données n'ont pas de valeurs renseignées pour les cibles.
Nous allons supprimer ces données inexploitables.

In [0]:
def missing_target(dataframe, target):
    """This function takes a dataframe and target feature, and returns
    a view of the dataframe containing the rows where the target value is missing.
    It also remove these rows from the original dataframe."""
    
    # Mask for rows where the target value is missing
    mask = dataframe[target].isnull()
    
    # Applying the mask to the dataframe and saving it as a copy
    result = dataframe[mask].copy()
    
    # Deleting the rows with missing target from the dataframe
    dataframe.dropna(subset=[target], inplace=True)
    
    # Returning a dataframe with rows where target is missing
    return result

In [48]:
target_cols = ['SiteEnergyUse(kBtu)','TotalGHGEmissions']
dataframe = data

# Initializing
production_set = pd.DataFrame()

# concatenating rows with missing values on any target
for target in target_cols:
    production_set = pd.concat([
        production_set,
        missing_target(dataframe, target)
    ])

# dropping duplicates rows (missing several targets)
production_set = production_set.drop_duplicates(keep='first').copy()

# Display the 'production set' on which to apply the ML model
production_set

,BuildingType,PrimaryPropertyType,CouncilDistrictCode,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions,DefaultData,ComplianceStatus,Latitude,Longitude,BuildingAge,SteamUse,NaturalGas,OtherFuelUse,FloorsGFA
